In [ ]:
class Game:
    def __init__(game_type, start_pos):
        self.game_type = game_type
        self.pos = start_pos
        
class GameSum():
    """ Calculates the next best move for a sum of games """
    
    def __init__(self, game_calculators):
        self.game_calculators = game_calculators
        
    def best_move(games):
        """ 
        Determines the next best move for a set of games
        games: collection of Games 
        returns: collection of Games with the best legal move made
        """
        return games # stub
    
    def visualize_game(games):
        """
        Prints visualization of sum of games to console
        """
        
        print("unimplemented")
        return
        